In [2]:
!pip install kagglehub

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "karkavelrajaj/amazon-sales-dataset",
    "amazon.csv"
)

print(df.shape)
print(df.head())
print(df.info())


/tmp/ipython-input-37328639.py:4: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 1.95M/1.95M [00:00<00:00, 70.5MB/s]

Extracting zip of amazon.csv...


(1465, 16)
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage rating rating_count  \
0       ₹1,099                 64%    4.2       24,269   
1         ₹349           

In [4]:
!pip install requests pandas numpy

In [5]:
import requests
import pandas as pd
import numpy as np


# link to another dataset ** ---- https://amazon-reviews-2023.github.io/#quick-start ---- **
# dataset used ** ---- https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset ---- **
# df = pd.read_csv('amazon.csv') # Kaggle Dataset

# print(df.shape, "\n")
# # 1465 X 16

# print(df.head(), "\n")
# print(df.info())




In [6]:
# Finding the missing parts of the dataset

missingCount = df.isna().sum()
print(missingCount, "\n")

missingPerc = df.isna().mean() * 100
print("The percentage of mising values in each columns is : \n\n",round(missingPerc, 2))




product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64 

The percentage of mising values in each columns is : 

 product_id             0.00
product_name           0.00
category               0.00
discounted_price       0.00
actual_price           0.00
discount_percentage    0.00
rating                 0.00
rating_count           0.14
about_product          0.00
user_id                0.00
user_name              0.00
review_id              0.00
review_title           0.00
review_content         0.00
img_link               0.00
product_link           0.00
dtype: float64


In [7]:
# Finding noise and inconsistencies and fixing the datatypes

df = df.dropna(axis = 0)

df["product_id"] = (df["product_id"].astype(str))
df["product_name"] = (df["product_name"].astype(str))
df["category"] = (df["category"].astype(str))

#   for numerical data, conversion to float
df["discounted_price"] = (df["discounted_price"].astype(str).str.replace(r"[^\d.]","", regex = True).astype(float))
df["actual_price"] = (df["actual_price"].astype(str).str.replace(r"[^\d.]","", regex = True).astype(float))
df["discount_percentage"] = (df["discount_percentage"].astype(str).str.replace(r"[^\d.]","", regex = True).astype(float))
df["rating"] = (df["rating"].astype(str).str.replace(r"[^\d.]","", regex = True).replace("", np.nan).astype(float))
df["rating_count"] = (df["rating_count"].astype(str).str.replace(r"[^\d.]","", regex = True).replace("", np.nan).astype(float))

df["about_product"] = (df["about_product"].astype(str))
df["user_id"] = (df["user_id"].astype(str))
df["user_name"] = (df["user_name"].astype(str))
df["review_id"] = (df["review_id"].astype(str))
df["review_title"] = (df["review_title"].astype(str))
df["img_link"] = (df["img_link"].astype(str))
df["product_link"] = (df["product_link"].astype(str))



In [8]:
# Finding the active links

#sample = df.sample(50, random_state = 0)

sample = df

result = []

for idx, url in sample["img_link"].items():
    try:
        res = requests.get(url, timeout = .1)
        stat = (res.status_code == 200) and ("image" in res.headers.get("Content-Type", ""))
        result.append((idx, int(stat)))

    except:
        result.append((idx, 0))

df["img_available"] = 0
for idx, stat in result:
    df.loc[idx, "img_available"] = stat

success = sum(stat for _, stat in result)
print(f"\n{success} of 1465 images were accessible.\n")

print(sample.dtypes)


844 of 1465 images were accessible.

product_id              object
product_name            object
category                object
discounted_price       float64
actual_price           float64
discount_percentage    float64
rating                 float64
rating_count           float64
about_product           object
user_id                 object
user_name               object
review_id               object
review_title            object
review_content          object
img_link                object
product_link            object
img_available            int64
dtype: object


In [9]:
!pip install transformers accelerate

In [10]:
import torch
from PIL import Image
from io import BytesIO
from tqdm import tqdm
from transformers import CLIPModel, CLIPProcessor


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [29]:
def clipSimilarityFunction(url, text):
  try:
    req = requests.get(url, timeout = 1)
    if(req.status_code != 200):
      return None
    img = Image.open(BytesIO(req.content))
    img = img.convert("RGB")
  except:
    return None

  inputs = processor(text = text, images = img, return_tensors = 'pt', padding = True, truncation = True, max_length = 77)
  inputs = {key: value.to(device) for key, value in inputs.items()}

  with torch.no_grad():
    outputs = model(**inputs)
    imgEmbed = outputs.image_embeds
    textEmbed = outputs.text_embeds

    imgEmbed = imgEmbed / imgEmbed.norm(dim = -1, keepdim = True)
    textEmbed = textEmbed / textEmbed.norm(dim = -1, keepdim = True)

    return (imgEmbed @ textEmbed.T).item()


In [30]:
sample = df[df["img_available"] == 1].copy()

sample["clip_similarity"] = None

for idx in tqdm(sample.index):
    sample.at[idx, "clip_similarity"] = clipSimilarityFunction(sample.at[idx, "img_link"], sample.at[idx, "product_name"])

100%|██████████| 844/844 [05:31<00:00,  2.55it/s]


In [34]:
sample["clip_similarity"] = sample["clip_similarity"].astype(float)
sample["clip_similarity"].describe()

,clip_similarity
count,844.000000
mean,0.323548
std,0.030981
min,0.204716
25%,0.306321
50%,0.323841
75%,0.341359
max,0.418573
